In [1]:
"""
import os
import shutil

# shutil.rmtree,递归删除
# os.symlink（src, dst），创建软链接，src -- 源地址， dst -- 目标地址

train_filenames = os.listdir("C:/Users/hh/Desktop/Kaggle/2.0 CatdogNet/input/train")
train_cat = filter(lambda x:x[:3] == "cat", train_filenames)
train_dog = filter(lambda x:x[:3] == "dog", train_filenames)

def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

# 使用os.mkdir()函数创建目录（创建一级目录）
rmrf_mkdir("C:/Users/hh/Desktop/Kaggle/2.0 CatdogNet/input/train4")
os.mkdir("C:/Users/hh/Desktop/Kaggle/2.0 CatdogNet/input/train4/cat")
os.mkdir("C:/Users/hh/Desktop/Kaggle/2.0 CatdogNet/input/train4/dog")

rmrf_mkdir("C:/Users/hh/Desktop/Kaggle/2.0 CatdogNet/input/test4")
os.symlink("C:/Users/hh/Desktop/Kaggle/2.0 CatdogNet/input/test",
           "C:/Users/hh/Desktop/Kaggle/2.0 CatdogNet/input/test4/test")
           
for filename in train_cat:
    os.symlink("C:/Users/hh/Desktop/Kaggle/2.0 CatdogNet/input/train" + filename, 
              "C:/Users/hh/Desktop/Kaggle/2.0 CatdogNet/input/train4/cat" + filename)
for filename in train_dog:
    os.symlink("C:/Users/hh/Desktop/Kaggle/2.0 CatdogNet/input/train" + filename,
              "C:/Users/hh/Desktop/Kaggle/2.0 CatdogNet/input/train4/cat" + filename)
              
"""

OSError: symbolic link privilege not held

In [5]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3

import h5py

def write_gap(MODEL, image_size, lambda_func = None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height,width,3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor = x, weights = "imagenet", include_top = False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    
    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("C:/Users/hh/Desktop/Kaggle/2.0 CatdogNet/input/train4", 
                                             image_size,shuffle = False, batch_size = 16)
    test_generator = gen.flow_from_directory("C:/Users/hh/Desktop/Kaggle/2.0 CatdogNet/input/test4", 
                                            image_size,shuffle = False, batch_size = 16, class_mode = None)
    
    train = model.predict_generator(train_generator, train_generator.nb_sample)
    test = model.predict_generator(test_generator, test_generator.nb_sample)
    
    with h5py.File("gap_%s.h5"%MODEL.func_name) as h:
        h.create_dataset("train",data = train)
        h.create_dataset("test",data = test)
        h.create_dataset("label",data = train_generator.classes)
        
write_gap(ResNet50,(224,224))

C:\Users\hh\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 25000 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


AttributeError: 'DirectoryIterator' object has no attribute 'nb_sample'

In [ ]:
write_gap(Xception, (299, 299), xception.preprocess_input)